In [3]:
import os
import shutil


def copy_segmentation_data(data_folder, fcn_folder):
    # Define file patterns
    rgb_pattern = "rgb_*.png"
    segmentation_image_pattern = "semantic_segmentation_*.png"
    segmentation_label_pattern = "semantic_segmentation_labels_*.png"

    # Create the fcn_data directory if it doesn't exist
    fcn_data_folder = os.path.join(fcn_folder, "fcn_data")
    imagesFolder = os.path.join(fcn_data_folder, "images")
    masksFolder = os.path.join(fcn_data_folder, "masks")
    labelsFolder = os.path.join(fcn_data_folder, "labels")

    os.makedirs(fcn_data_folder, exist_ok=True)
    os.makedirs(imagesFolder, exist_ok=True)
    os.makedirs(masksFolder, exist_ok=True)
    os.makedirs(labelsFolder, exist_ok=True)
    counter = 0
    # Copy RGB images
    for file in sorted(os.listdir(data_folder)):
        if file.startswith("rgb_") and file.endswith(".png"):
            shutil.copy(os.path.join(data_folder, file), imagesFolder)
            counter = counter + 1

        if counter == 100:
            break
    counter = 0
    # Copy segmentation images
    for file in sorted(os.listdir(data_folder)):
        if file.startswith("semantic_segmentation_") and file.endswith(".png"):
            shutil.copy(os.path.join(data_folder, file), masksFolder)
            counter = counter + 1

        if counter == 100:
            break
    counter = 0
    # Copy segmentation labels
    for file in sorted(os.listdir(data_folder)):
        if file.startswith("semantic_segmentation_labels_") and file.endswith(".json"):
            shutil.copy(os.path.join(data_folder, file), labelsFolder)
            counter = counter + 1

        if counter == 100:
            break


# Example usage
data_folder = "../data"
fcn_folder = "./"
copy_segmentation_data(data_folder, fcn_folder)

# Read all labels

In [2]:
import os
import json

# Directory containing the JSON files
directory = "./fcn_data/labels"
map = {}
# Iterate through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".json"):
        file_path = os.path.join(directory, filename)

        # Load the JSON data
        with open(file_path) as f:
            labels = json.load(f)

        # Create a dictionary to hold class and corresponding color
        class_color_mapping = {value["class"]: key for key, value in labels.items()}

        # Print the class and corresponding color for the current file
        for class_name, color in class_color_mapping.items():
            if class_name not in map.keys():
                map[class_name] = color
            elif color != map[class_name]:
                print("warning")

In [6]:
map

{'BACKGROUND': '(0, 0, 0, 0)',
 'UNLABELLED': '(0, 0, 0, 255)',
 'iwhub': '(25, 255, 82, 255)',
 'dolly': '(25, 82, 255, 255)',
 'pallet': '(255, 25, 197, 255)',
 'rack': '(140, 255, 25, 255)',
 'railing': '(255, 111, 25, 255)',
 'floor': '(226, 255, 25, 255)',
 'forklift': '(255, 197, 25, 255)',
 'crate': '(140, 25, 255, 255)',
 'stillage': '(54, 255, 25, 255)'}